## ReAct Agent
ReAct is a pattern combining Reasoning + Acting: the model reasons (plans) in steps, then performs utility actions (API queries, searches, calculations), and the resulting observations are incorporated into subsequent reasoning steps.

### Install libraries

In [1]:
!pip install -q langchain langchain-openai python-dotenv

### Prepare model

In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("Model ready.")

Model ready.


### How does the model use tools?

In [3]:
import re

#  tool - tool
def greet_user(name: str) -> str:
    """A function that greets the user by name."""
    return f"Hello, {name}! Nice to meet you!"

prompt = """
Answer the user's question.
If the user introduces themselves, use the format: GREET[name] and do not use the user's name again.

Question: Hi, my name is Michał and I'd like to know which came first - the egg or the nest?
"""

response = llm.invoke(prompt)
response_text = response.content

pattern = r'GREET\\[(\\w+)\\]'
match = re.search(pattern, response_text)

# If pattern found, call the tool-function
if match:
    name = match.group(1)
    print("GREET pattern detected, calling function-tool...")
    result = greet_user(name)
    print(f"Function result: {result}")
    response_text = response_text.replace(match.group(0), result)
else:
    print("GREET pattern not found in response.")

print("LLM response:")
print(response_text)

GREET pattern not found in response.
LLM response:
GREET[Michał] The egg came first. Eggs have been around for hundreds of millions of years, laid by species that existed long before birds and their nests evolved.


### Define tools

In [4]:
from langchain_classic.tools import tool

@tool
def add_numbers(**kwargs:int) -> int:
    """Add numbers together."""
    return sum(kwargs['kwargs'].values())

@tool
def company_info(name: str) -> str:
    "return info about company"
    data = {
        "OpenAI": "An artificial intelligence company, creator of ChatGPT. 10 years old.",
        "LangChain": "A framework for building AI applications with LLM. 4 years old.",
        "Tesla": "An electric car manufacturer. 23 years old."
    }
    return data.get(name, "I have no information about this company.")

tools = [add_numbers, company_info]

### ReAct Agent

In [5]:
from langchain_classic.agents import initialize_agent, AgentType

agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=6
)

agent.run("Tell me what LangChain, Tesla, and OpenAI do. Then tell me the total number of years these three companies have been in business.")

/tmp/ipykernel_18659/3571289280.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the [LangGraph documentation](https://langchain-ai.github.io/langgraph/) as well as guides for [Migrating from AgentExecutor](https://python.langchain.com/docs/how_to/migrate_agent/) and LangGraph's [Pre-built ReAct agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).
  agent = initialize_agent(
/tmp/ipykernel_18659/3571289280.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  agent.run("Tell me what LangChain, Tesla, and OpenAI do. Then tell me the total number of years these three compani



> Entering new AgentExecutor chain...
Thought: I will first gather information about each company: LangChain, Tesla, and OpenAI. Then, I will calculate the total number of years they have been in business.
Action:
```
{
  "action": "company_info",
  "action_input": {
    "name": "LangChain"
  }
}
```
Observation: A framework for building AI applications with LLM. 4 years old.
Thought:To answer your question, I will gather information about Tesla and OpenAI, and then calculate the total number of years these companies have been in business.

Action:
```
{
  "action": "company_info",
  "action_input": {
    "name": "Tesla"
  }
}
```

Observation: An electric car manufacturer. 23 years old.
Thought:Action:
```
{
  "action": "company_info",
  "action_input": {
    "name": "OpenAI"
  }
}
```

Observation: An artificial intelligence company, creator of ChatGPT. 10 years old.
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "LangChain is a framework for building AI applic

'LangChain is a framework for building AI applications with large language models (LLM) and is 4 years old. Tesla is an electric car manufacturer and is 23 years old. OpenAI is an artificial intelligence company, known for creating ChatGPT, and is 10 years old. The total number of years these three companies have been in business is 37 years.'